# Adding Model Prediction to a Voxel51 Dataset
adapted from: https://voxel51.com/docs/fiftyone/recipes/model_inference.html

If you haven’t already, install FiftyOne:

In [ ]:
!pip install fiftyone

# Build FiftyOne Dataset

In [ ]:
#config
import fiftyone as fo
import os
import pandas as pd


file_path = "/tf/testing"
planes = pd.read_csv("./aircraftDatabase.csv",index_col='icao24')
#print(planes)
batch_size = 1000
# Create the dataset
try:
    dataset = fo.load_dataset("plane-dataset")
except fo.core.dataset.DoesNotExistError:
    dataset = fo.Dataset(name="plane-dataset")
    


In [ ]:

def buildImageList(filePath):
    labelbox_import = []
    for folder, subfolders, files in os.walk(filePath):
        for file in files:
            if file.endswith(".jpg"):
                image_filename = os.path.basename(file)
                external_id = os.path.splitext(image_filename)[0]
                image_path = os.path.abspath(os.path.join(folder, file))

                item = {"file_path": image_path,
                    "external_id": external_id}
                labelbox_import.append(item)
    return labelbox_import

def importImageList(fileList):

    samples = []
    for image in fileList:
        plane_id = image["external_id"].split("_")[0]
        
        
        sample = fo.Sample(filepath=image["file_path"])
        sample["icao24"] = fo.Classification(label=plane_id.lower())
        try:
            plane = planes.loc[ plane_id.lower()]
            if plane.size == 27:
                #print("Adding metadata for plane {} onto image {}".format(plane["icao24"].values[0], image["external_id"]))
                if isinstance(plane["model"].values[0], str):
                    sample["model"] = fo.Classification(label=plane["model"].values[0] )
                if isinstance(plane["manufacturername"].values[0], str):
                    sample["manufacturer"] = fo.Classification(label=plane["manufacturername"].values[0])
        except KeyError:
            print("DB entry not found for: {}".format(plane_id))
        sample.save()
        samples.append(sample)


    dataset.add_samples(samples)
    dataset.persistent = True
    # View summary info about the dataset
    print(dataset)

    # Print the first few samples in the dataset
    print(dataset.head())

In [ ]:
image_list = buildImageList(file_path)
if len(image_list) > 0:
    print("Found {} images, processing in batches of: {}".format(len(image_list), batch_size))

    for i in range(0, len(image_list), batch_size):
        chunk = image_list[i:i + batch_size]
        importImageList(chunk)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# image folder
folder_path = '/tf/testing'
# path to model
model_path = '/tf/dataset/plane-detector'
# dimensions of images
img_width, img_height = 299, 299


In [ ]:
# load the trained model
model = load_model(model_path)
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)


In [ ]:
print(dataset)

In [ ]:
#view = dataset.limit(100)
for sample in dataset: #view:
    image_size = (img_width, img_height)
    img = keras.preprocessing.image.load_img(sample.filepath, target_size=image_size)

    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis

    predictions = model.predict(img_array)
    score = predictions[0][0]
    #print(score)
    if score>0:
        label="plane"
        sample.tags.append("plane")
    else:
        label="no plane"
        sample.tags.append("noPlane")
    sample["plane"] = fo.Classification(confidence=score,label=label )
    sample.save()


In [ ]:
view = dataset.exists("plane")
session.view = view